# Ajuste fino del modelo `tiny-llama`

**ESTE NOTEBOOK NO FUNCIONARÁ EN UN MAC OS POR LOS REQUISITOS TÉCNICOS DEL TRAINER**

In [ ]:
from huggingface_hub import login
token = "huggingface_token"
print("Hugging Face logging")
login(token)

Hugging Face logging


In [23]:
import torch
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")


Usando dispositivo: cuda


In [24]:
!pip install datasets trl

## Cargar el modelo + tokenizador

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/phi-2"
cache_dir = "./models/microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float32 if device == "mps" else (torch.float16 if torch.cuda.is_available() else torch.float32),
            cache_dir=cache_dir,
            local_files_only=False,
        ).to(device)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

## Cargar el dataset y preprocesar los datos

In [58]:
import torch
from datasets import load_dataset

tokenizer.pad_token = tokenizer.eos_token  # Asignar el token de padding

# Load local dataset
# Change the path and format according to your data file
# Supported formats: csv, json, txt, parquet
dataset = load_dataset('json', data_files={
    'train': 'medicare/medicare_110k_train.json',
    'test': 'medicare/medicare_110k_test.json'
})['train']  # We only use train for fine-tuning

def preprocess_function(examples):
    max_length = 2048  # Máxima longitud para Phi-2

    # Convertir conversaciones al formato esperado
    conversations = []
    for text in examples['Conversation']:
        dialogue = []
        parts = text.split('[|')
        for part in parts[1:]:  # Saltar el primer elemento vacío
            role, content = part.split('|]')
            role = role.strip().lower()
            content = content.strip()
            if role == 'human':
                dialogue.append({"role": "user", "content": content})
            elif role == 'ai':
                dialogue.append({"role": "assistant", "content": content})
    
        # Aplicar el chat template para formatear el diálogo
        formatted_text = tokenizer.apply_chat_template(
            dialogue,
            tokenize=False,
            return_tensors=None
        )
        conversations.append(formatted_text)
    
    # Tokenizar las conversaciones formateadas
    tokenized = tokenizer(
        examples["Conversation"],
        padding="max_length",  # O usa "longest" si tienes más memoria
        truncation=True,
        max_length=512,  # Ajusta según el modelo
        return_tensors="pt",
    )
    return tokenized

# Aplicar preprocesamiento al dataset
dataset = dataset.map(preprocess_function, batched=True)

print(dataset)

Map:   0%|          | 0/106556 [00:00<?, ? examples/s]

Dataset({
    features: ['Conversation', 'input_ids', 'attention_mask'],
    num_rows: 106556
})


## Configurar y ejecutar el entrenamiento

Configurar PEFT (ajuste fino eficiente de parámetros)

In [59]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Solución para mejorar tokenización y manejo de memoria
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
import torch

# Configuración de entrenamiento
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    remove_unused_columns=False,  # Asegura que se pasen las columnas correctas
    output_dir="./output",
    logging_steps=10,
    max_steps=1000,
    learning_rate=2e-4,
    fp16=True,  # Usa FP16 si tienes compatibilidad con CUDA
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    data_collator=data_collator,
    args=training_args,
)

# Entrenamiento
trainer.train()
trainer.save_model("./models/phi-2-FT")


AttributeError: 'function' object has no attribute 'config'

# Uso de los modelos

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class DialogueController:

    def __init__(self, system_prompt, assistant_token="<|assistant|>", keep_cronology=True):
        self.system_prompt = system_prompt
        self.prompt = [ { "role": "system", "content": system_prompt} ]
        self.assistant_token = assistant_token
        self.keep_cronology = keep_cronology

    def get_prompt(self):
        return self.prompt

    def add_user_prompt(self, user_prompt, role="user"):
        user_prompt_json = { "role": role, "content":  user_prompt}
        if self.keep_cronology:
          self.prompt.append(user_prompt_json)
        else:
          self.prompt = [self.prompt[0], user_prompt_json]
        return self.prompt

    def add_assistant_prompt(self, assistant_prompt):
        if self.keep_cronology:
          assistant_prompt_exclusive = assistant_prompt.split(self.assistant_token)
          final_prompt = ""
          if len(assistant_prompt_exclusive) == 2:
              final_prompt = assistant_prompt_exclusive[1]
          self.add_user_prompt(final_prompt, role="assistant")


class Chatbot2:

    def __init__(self, controller, chatbot_model="microsoft/phi-2", cache_dir="./microsoft/phi-2"):
        global device
        self.tokenizer = AutoTokenizer.from_pretrained(chatbot_model)
        self.model = AutoModelForCausalLM.from_pretrained(
            chatbot_model,
            torch_dtype=torch.float32 if device == "mps" else (torch.float16 if torch.cuda.is_available() else torch.float32),
            cache_dir=cache_dir,
            local_files_only=False,
        ).to(device)
        self.model_device = device
        self.dialogue_controller = controller

    def __run_prompt(self, prompt, do_sample, temperature, top_p, max_length, show_prompt):
        formatted_prompt = self.tokenizer.apply_chat_template(conversation=prompt, tokenize=False, return_dict=False, add_generation_prompt=True)

        # Tokenizar
        inputs = self.tokenizer(formatted_prompt, max_length=max_length, truncation=True, return_tensors="pt")
        inputs = {k: v.to(self.model_device) for k, v in inputs.items()}

        # Muestra infomacion de log
        if show_prompt:
          print(formatted_prompt)
          print("--- Token size: ---")
          [print("\t", k, ": ", len(v[0])) for k, v in inputs.items()]
          print("-------------------")

        # Generar respuesta
        outputs = self.model.generate(
            **inputs,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            pad_token_id=self.tokenizer.eos_token_id
        )

        # Decodificar y limpiar respuesta
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


    def answer(self, user_prompt, do_sample=True, temperature=0.1, top_p=0.9, max_length=2047, show_prompt=False):
        # Actualiza el prompt
        prompt = self.dialogue_controller.add_user_prompt(user_prompt)
        # Resolver prompt
        answer = self.__run_prompt(prompt, do_sample, temperature, top_p, max_length, show_prompt)
        # Actualiza el prompt con la respuesta del asistente
        self.dialogue_controller.add_assistant_prompt(answer)
        return answer



In [14]:
# Modelos disponibles
fine_tuned_model = "./models/phi-2-FT"
model_name = "microsoft/phi-2"
cache_dir = "./models/microsoft/phi-2"

# Cargar los modelos a través del chatbot
dialog_controller1 = DialogueController(system_prompt="Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'.",keep_cronology=False)
dialog_controller2 = DialogueController(system_prompt="Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'.",keep_cronology=False)

chatbot_finetuned = Chatbot2(controller=dialog_controller1, chatbot_model=model_name, cache_dir=fine_tuned_model)
chatbot_standard = Chatbot2(controller=dialog_controller2)

questions = [
    ("Me duele la cabeza, ¿qué puedo hacer?", "Debes tomar un analgésico y descansar."),
    ("He comido algo en mal estado, ¿qué debo hacer?", "Debes acudir a un médico."),
    ("¿Tengo un esguince en la rodilla, qué debo hacer?", "Debes aplicar hielo y reposar."),
    ("¿Qué debo hacer si tengo fiebre?", "Debes tomar un antipirético."),
    ("¿Qué es la diabetes?", "La diabetes es una enfermedad crónica que afecta a la forma en que el cuerpo convierte los alimentos en energía."),
    ("¿Qué es la hipertensión?", "La hipertensión es una enfermedad crónica que se caracteriza por una presión arterial elevada."),
    ("¿Qué es el colesterol?", "El colesterol es una sustancia cerosa que se encuentra en la sangre."),
    ("¿Qué es el cáncer?", "El cáncer es una enfermedad en la que las células anormales se dividen sin control y pueden invadir otros tejidos."),
    ("¿Qué es la artritis?", "La artritis es una enfermedad que causa dolor e inflamación en las articulaciones."),
    ("¿Qué es la osteoporosis?", "La osteoporosis es una enfermedad en la que los huesos se vuelven frágiles y propensos a fracturas."),
    ("¿Qué es el asma?", "El asma es una enfermedad crónica que afecta las vías respiratorias."),
    ("¿Qué es la migraña?", "La migraña es un tipo de dolor de cabeza que puede ser intenso y debilitante."),
    ("¿Qué es la depresión?", "La depresión es un trastorno del estado de ánimo que causa una sensación persistente de tristeza y pérdida de interés."),
    ("¿Qué es la ansiedad?", "La ansiedad es una emoción de miedo o preocupación excesiva."),
    ("¿Qué es el estrés?", "El estrés es una respuesta física y emocional a un estímulo que causa tensión."),
    ("¿Qué es el Alzheimer?", "El Alzheimer es una enfermedad cerebral progresiva que afecta la memoria y la capacidad de pensar."),
    ("¿Qué es el Parkinson?", "El Parkinson es una enfermedad del sistema nervioso que afecta el movimiento."),
    ("¿Qué es la esclerosis múltiple?", "La esclerosis múltiple es una enfermedad autoinmune que afecta el sistema nervioso."),
]
for question, answer in questions:
    print("regular: ",chatbot_standard.answer(user_prompt=question))
    print("fine tuned: ",chatbot_finetuned.answer(user_prompt=question))
    print("****expected answer: ", answer)
    print("----------------------")



regular:  <|system|>
Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'. 
<|user|>
Cuál es la capital de francia? 
<|assistant|>
La capital de Francia es París.
fine tuned:  <|system|>
Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'. 
<|user|>
Cuál es la capital de francia? 
<|assistant|>
La capital de Francia es París.
****expected answer:  Paris
----------------------
regular:  <|system|>
Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'. 
<|user|>
¿Qué juegos me recomendarías si me ha gustado Undertale? 
<|assistant|>
Sure, Undertale is a great game to play if you enjoyed the aesthetic and gameplay of Toby Fox's Undertale. Here are some games that you might enjoy:

1. Celeste: This game is a platformer with a unique twist - the gameplay is based on the player's breathing. It's a